In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from tensorflow import keras

from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from keras.layers import Dense, SimpleRNN, RNN, LSTM,Dropout,Bidirectional,Attention,Embedding,BatchNormalization
from keras.optimizers import Adam,SGD,RMSprop,schedules
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
# load dataset
dataframe = pd.read_csv("hackaton.csv", header=None)
# convert to numpy
dataset = dataframe.values
# split into input (X) and output (Y) variables, skip Header row
X = dataset[1:,0:8]
Y = dataset[1:,8].astype(float)

In [3]:
enc=OneHotEncoder()
enc.fit(X)
xhot = enc.transform(X).toarray()
#xhot = OneHotEncoder().fit_transform(X).toarray()

In [ ]:
xhot[0].shape[0]
#enc.categories_

In [6]:
#Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
y_train = scaler.fit_transform(Y.reshape(-1, 1))


array([[0.0401837 ],
       [0.04196423],
       [0.06070973],
       ...,
       [0.07688659],
       [0.23183342],
       [0.09999465]])

In [7]:
x_train=xhot
#Convert the x_train and y_train to numpy arrays, Keras needs it for model.fit 
x_train, y_train = np.array(x_train), np.array(y_train)

In [8]:
FC = Sequential()
FC.add(Dense(xhot[0].shape[0], activation='relu', input_shape=[xhot[0].shape[0]]))
FC.add(Dropout(0.2))
FC.add(Dense(xhot[0].shape[0],activation='relu'))
FC.add(Dropout(0.2))
FC.add(Dense(xhot[0].shape[0], activation='relu'))
FC.add(Dropout(0.2))
FC.add(Dense(1, activation='linear'))

In [ ]:
FC.summary()

In [9]:
#Compile the model
lr_schedule = schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.8)
opt = RMSprop(learning_rate=lr_schedule)
#opt = RMSprop(learning_rate=4e-3)
FC.compile(optimizer=opt, loss='mean_squared_error')

In [10]:
#Train the model
FC_info=FC.fit(x_train, y_train, batch_size=1, epochs=100,verbose=1, shuffle=False, validation_split=0.1)

Epoch 1/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0162 - val_loss: 0.0027
Epoch 2/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0091 - val_loss: 0.0018
Epoch 3/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0090 - val_loss: 0.0017
Epoch 4/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0090 - val_loss: 0.0015
Epoch 5/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0079 - val_loss: 0.0015
Epoch 6/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0071 - val_loss: 0.0016
Epoch 7/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0070 - val_loss: 0.0015
Epoch 8/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0067 - val_loss: 0.0015
Epoch 9/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0066 - val_loss: 0.0020
Epoch 10/100
2097/2097 [==============================] - 8s 4ms/step - l

Epoch 80/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0029 - val_loss: 0.0038
Epoch 81/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0031 - val_loss: 0.0035
Epoch 82/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0032 - val_loss: 0.0037
Epoch 83/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0028 - val_loss: 0.0035
Epoch 84/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0031 - val_loss: 0.0036
Epoch 85/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0028 - val_loss: 0.0039
Epoch 86/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0030 - val_loss: 0.0040
Epoch 87/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0029 - val_loss: 0.0034
Epoch 88/100
2097/2097 [==============================] - 8s 4ms/step - loss: 0.0030 - val_loss: 0.0037
Epoch 89/100
2097/2097 [==============================] - 8s 4ms

In [12]:
ohe=enc.transform([['CPU 3a','RAM 4','S 6','GPU 1','15','HD+','Non-Touch','DE']]).toarray()

In [ ]:
ohe.shape

In [13]:
pred_price = FC.predict(ohe)
#undo the scaling 
pred_price = scaler.inverse_transform(pred_price)
print(pred_price[0][0])
print(Y[0])

697.47034
447.22
